In [1]:
import pandas as pd
import pymongo
import selenium
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

In [2]:
# executable_path = {'executable_path': '../static/geckodriver'}
# browser = Browser('firefox', **executable_path, headless=True)
executable_path = {'executable_path': '../static/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
event_df = pd.read_csv("../data/rolling_event_data.csv")
event_df["date"] = event_df["date"].str.strip("DATE:")
event_df["location"] = event_df["location"].str.strip("LOCATION:")
event_df = event_df.drop(columns="Unnamed: 0")
event_df

,event,date,location,event_url
0,UFC Fight Night: Ladd vs. Dumont,"October 16, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/8f4616698508...
1,UFC Fight Night: Dern vs. Rodriguez,"October 09, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/d247691a6c0e...
2,UFC Fight Night: Santos vs. Walker,"October 02, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/e15d0a2519d6...
3,UFC 266: Volkanovski vs. Ortega,"September 25, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/0eec866a0778...
4,UFC Fight Night: Smith vs. Spann,"September 18, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/4c27ca8c8481...
...,...,...,...,...
575,UFC 6: Clash of the Titans,"July 14, 1995","Casper, Wyoming, US",http://ufcstats.com/event-details/1c3f5e85b59e...
576,UFC 5: The Return of the Beast,"April 07, 1995","Charlotte, North Carolina, US",http://ufcstats.com/event-details/dedc3bb440d0...
577,UFC 4: Revenge of the Warriors,"December 16, 1994","Tulsa, Oklahoma, US",http://ufcstats.com/event-details/b60391da771d...
578,UFC 3: The American Dream,"September 09, 1994","Charlotte, North Carolina, US",http://ufcstats.com/event-details/1a49e0670dfa...


In [5]:
event_urls = event_df["event_url"].tolist()

In [6]:
fight_urls_df = pd.read_csv("../data/rolling_fight_urls.csv")
fight_urls_df = fight_urls_df.drop(columns="Unnamed: 0")
fight_urls_df

,url
0,http://ufcstats.com/fight-details/330e67abac6e...
1,http://ufcstats.com/fight-details/188b63ec6ee1...
2,http://ufcstats.com/fight-details/d9ca00b3c62a...
3,http://ufcstats.com/fight-details/b193e2317b21...
4,http://ufcstats.com/fight-details/a2e1116a9fe8...
...,...
6296,http://ufcstats.com/fight-details/3b020d4914b4...
6297,http://ufcstats.com/fight-details/d917c8c7461b...
6298,http://ufcstats.com/fight-details/ccee020be2e8...
6299,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [7]:
fight_urls = fight_urls_df["url"].tolist()
fight_urls

['http://ufcstats.com/fight-details/330e67abac6e5ce7',
 'http://ufcstats.com/fight-details/188b63ec6ee1c1c9',
 'http://ufcstats.com/fight-details/d9ca00b3c62a874d',
 'http://ufcstats.com/fight-details/b193e2317b21125c',
 'http://ufcstats.com/fight-details/a2e1116a9fe84aa9',
 'http://ufcstats.com/fight-details/fefa96b6d939ffea',
 'http://ufcstats.com/fight-details/e3881a6895be3e9e',
 'http://ufcstats.com/fight-details/be8bee783d22bee2',
 'http://ufcstats.com/fight-details/cf020c71f1dbfa0b',
 'http://ufcstats.com/fight-details/814d481e79e9a525',
 'http://ufcstats.com/fight-details/149d6b90f33634a4',
 'http://ufcstats.com/fight-details/d5ffdb5388ffabc6',
 'http://ufcstats.com/fight-details/20f19439e9b100b0',
 'http://ufcstats.com/fight-details/38957d8eb185391c',
 'http://ufcstats.com/fight-details/6101249c2cdd1493',
 'http://ufcstats.com/fight-details/777c65184aa63894',
 'http://ufcstats.com/fight-details/df16b9c2fc9722e9',
 'http://ufcstats.com/fight-details/f526892a27b50576',
 'http://u

In [8]:
all_events_url = "http://ufcstats.com/statistics/events/completed?page=all"

In [9]:
browser.visit(all_events_url)
all_events_soup = bs(browser.html, "html.parser")
past_events = all_events_soup.find_all(class_="b-link b-link_style_black")
future_events = all_events_soup.find_all(class_="b-link b-link_style_white")

new_event_urls = []
future_event_urls = []
for event in future_events:
    future_url = event["href"]
    if future_url not in future_event_urls:
        future_event_urls.append(future_url)
        
for url in past_events:
    past_url = url["href"]
    if past_url not in event_urls:
        new_event_urls.append(past_url)
new_event_urls

['http://ufcstats.com/event-details/8a9c6c4301f6d088']

In [10]:
future_event_urls

['http://ufcstats.com/event-details/3974fa35c917af1d']

In [11]:
future_events_data = []
future_fight_urls = []
for event in future_event_urls:
    browser.visit(event)
    event_soup = bs(browser.html, "html.parser")
    hrefs = event_soup.find_all(class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    
    
    event_date = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    event_name = browser.find_by_xpath("/html/body/section/div/h2/span").text
    event_location = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    
    event_info = {"event": event_name,
                 "date": event_date,
                 "location": event_location,
                 "event_url": event}
    
    future_events_data.append(event_info)
    
      
    for cols in hrefs:
        href = cols["data-link"]        
       
        if href not in fight_urls:
            future_fight_urls.append(href)
            
future_fight_urls

['http://ufcstats.com/fight-details/919fb76d2f3e5827',
 'http://ufcstats.com/fight-details/18edff5447989b3f',
 'http://ufcstats.com/fight-details/5ad4c0eae7512878',
 'http://ufcstats.com/fight-details/13e3e41e0c6f4e04',
 'http://ufcstats.com/fight-details/987cdcbc809e210d',
 'http://ufcstats.com/fight-details/58b19ba734fe93a3',
 'http://ufcstats.com/fight-details/d84968d7e9fcb4f3',
 'http://ufcstats.com/fight-details/41cdfea7df63003f',
 'http://ufcstats.com/fight-details/768c86e307295944',
 'http://ufcstats.com/fight-details/efc6af0151f72b87',
 'http://ufcstats.com/fight-details/92c49cb075f612d7',
 'http://ufcstats.com/fight-details/156f2425a937b22f',
 'http://ufcstats.com/fight-details/4a5e5232346993c7',
 'http://ufcstats.com/fight-details/4cdd5761107e464f',
 'http://ufcstats.com/fight-details/6163c18411aca830']

In [12]:
future_event_df = pd.DataFrame(future_events_data)
future_event_df["date"] = future_event_df["date"].str.strip("DATE:")
future_event_df["location"] = future_event_df["location"].str.strip("LOCATION:")
future_event_df

,event,date,location,event_url
0,UFC 267: Blachowicz vs. Teixeira,"October 30, 2021","Abu Dhabi, Abu Dhabi, United Arab Emirates",http://ufcstats.com/event-details/3974fa35c917...


In [13]:
future_event_df.to_csv("../data/future_event_data.csv")

In [14]:
new_events_data = []
new_fight_urls = []
for event in new_event_urls:
    browser.visit(event)
    event_soup = bs(browser.html, "html.parser")
    hrefs = event_soup.find_all(class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    
    
    event_date = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    event_name = browser.find_by_xpath("/html/body/section/div/h2/span").text
    event_location = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    
    event_info = {"event": event_name,
                 "date": event_date,
                 "location": event_location,
                 "event_url": event}
    
    new_events_data.append(event_info)
    
      
    for cols in hrefs:
        href = cols["data-link"]        
       
        if href not in fight_urls:
            new_fight_urls.append(href)
            
new_fight_urls

['http://ufcstats.com/fight-details/860be3cc82895177',
 'http://ufcstats.com/fight-details/9fe62ea683decc55',
 'http://ufcstats.com/fight-details/69167b996ce7a7e4',
 'http://ufcstats.com/fight-details/98842f17dfe4b4ef',
 'http://ufcstats.com/fight-details/587d373f18f6c8c0',
 'http://ufcstats.com/fight-details/e078d18fa9ed100d',
 'http://ufcstats.com/fight-details/88256dd5a9aef1cc',
 'http://ufcstats.com/fight-details/d44d735e5079bfb8',
 'http://ufcstats.com/fight-details/1fe58cdab57d233b',
 'http://ufcstats.com/fight-details/85f920746c58d0a6',
 'http://ufcstats.com/fight-details/6d7eecf4acecda2a',
 'http://ufcstats.com/fight-details/acec5f4c6fee1ae2',
 'http://ufcstats.com/fight-details/61d9eff5fa4c8578',
 'http://ufcstats.com/fight-details/7151ae256e6c181b']

In [15]:
new_event_df = pd.DataFrame(new_events_data)
new_event_df["date"] = new_event_df["date"].str.strip("DATE:")
new_event_df["location"] = new_event_df["location"].str.strip("LOCATION:")
new_event_df

,event,date,location,event_url
0,UFC Fight Night: Costa vs. Vettori,"October 23, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/8a9c6c4301f6...


In [16]:
event_frames = [new_event_df, event_df]
rolling_event_df = pd.concat(event_frames)
# rolling_event_df = rolling_event_df.drop(columns="Unnamed: 0")
rolling_event_df

,event,date,location,event_url
0,UFC Fight Night: Costa vs. Vettori,"October 23, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/8a9c6c4301f6...
0,UFC Fight Night: Ladd vs. Dumont,"October 16, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/8f4616698508...
1,UFC Fight Night: Dern vs. Rodriguez,"October 09, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/d247691a6c0e...
2,UFC Fight Night: Santos vs. Walker,"October 02, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/e15d0a2519d6...
3,UFC 266: Volkanovski vs. Ortega,"September 25, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/0eec866a0778...
...,...,...,...,...
575,UFC 6: Clash of the Titans,"July 14, 1995","Casper, Wyoming, US",http://ufcstats.com/event-details/1c3f5e85b59e...
576,UFC 5: The Return of the Beast,"April 07, 1995","Charlotte, North Carolina, US",http://ufcstats.com/event-details/dedc3bb440d0...
577,UFC 4: Revenge of the Warriors,"December 16, 1994","Tulsa, Oklahoma, US",http://ufcstats.com/event-details/b60391da771d...
578,UFC 3: The American Dream,"September 09, 1994","Charlotte, North Carolina, US",http://ufcstats.com/event-details/1a49e0670dfa...


In [17]:
rolling_event_df.to_csv("../data/rolling_event_data.csv")

In [18]:
fighter_data = []
full_fight_data = []

for fight_url in new_fight_urls:
    
    browser.visit(fight_url)
    print(fight_url)
    
    event_name = browser.find_by_xpath("/html/body/section/div/h2/a").text
    winner_method = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[1]/i[2]").text
    end_round = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[2]").text
    end_time = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[3]").text
    referee = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[5]/span").text
    weight_class = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/i").text
    
    fighter_1_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[1]/div/h3/a").text
    fighter_1_corner_color = "red"
    fighter_1_outcome = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[1]/i").text
    fighter_1_KDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[2]/p[1]").text
    fighter_1_sig_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[3]/p[1]").text
    fighter_1_total_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[5]/p[1]").text
    fighter_1_TDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[6]/p[1]").text
    fighter_1_sub_atts = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[8]/p[1]").text
    fighter_1_reversals = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[9]/p[1]").text
    fighter_1_ctrl_time = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[10]/p[1]").text
    fighter_1_sig_head = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[4]/p[1]").text
    fighter_1_sig_body = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[5]/p[1]").text
    fighter_1_sig_leg = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[6]/p[1]").text
    fighter_1_sig_distance = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[7]/p[1]").text
    fighter_1_sig_clinch = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[8]/p[1]").text
    fighter_1_sig_ground = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[9]/p[1]").text
    
    fighter_2_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[2]/div/h3/a").text
    fighter_2_corner_color = "blue"
    fighter_2_outcome = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[2]/i").text
    fighter_2_KDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[2]/p[2]").text
    fighter_2_sig_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[3]/p[2]").text
    fighter_2_total_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[5]/p[2]").text
    fighter_2_TDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[6]/p[2]").text
    fighter_2_sub_atts = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[8]/p[2]").text
    fighter_2_reversals = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[9]/p[2]").text
    fighter_2_ctrl_time = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[10]/p[2]").text
    fighter_2_sig_head = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[4]/p[2]").text
    fighter_2_sig_body = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[5]/p[2]").text
    fighter_2_sig_leg = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[6]/p[2]").text
    fighter_2_sig_distance = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[7]/p[2]").text
    fighter_2_sig_clinch = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[8]/p[2]").text
    fighter_2_sig_ground = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[9]/p[2]").text

    
    fighter_1_info = {"name" : fighter_1_name,
                  "corner" : fighter_1_corner_color,
                  "weight_class": weight_class,
                  "opponent": fighter_2_name,
                  "event": event_name,
                  "outcome" : fighter_1_outcome,
                  "method" : winner_method,
                  "round": end_round,
                  "Time": end_time,
                  "Significant_strikes": fighter_1_sig_str,
                  "Total_strikes": fighter_1_total_str,
                  "KDs": fighter_1_KDs,    
                  "TDs": fighter_1_TDs,
                  "Sub_Atts": fighter_1_sub_atts,
                  "Reversals": fighter_1_reversals,
                  "Control_time": fighter_1_ctrl_time,
                  "Sig_Str_head": fighter_1_sig_head,
                  "Sig_Str_head": fighter_1_sig_head,
                  "Sig_Str_body": fighter_1_sig_body,
                  "Sig_Str_leg": fighter_1_sig_leg,
                  "Sig_Str_distance": fighter_1_sig_distance,
                  "Sig_Str_clinch": fighter_1_sig_clinch,
                  "Sig_Str_ground": fighter_1_sig_ground,}
      
    fighter_2_info = {"name" : fighter_2_name,
                  "corner" : fighter_2_corner_color,
                  "weight_class": weight_class,
                  "opponent" : fighter_1_name,
                  "event" : event_name,
                  "outcome" : fighter_2_outcome,
                  "method" : winner_method,
                  "round": end_round,
                  "Time": end_time,
                  "Significant_strikes": fighter_2_sig_str,
                  "Total_strikes": fighter_2_total_str,
                  "KDs": fighter_2_KDs,
                  "TDs": fighter_2_TDs,
                  "Sub_Atts": fighter_2_sub_atts,
                  "Reversals": fighter_2_reversals,
                  "Control_time": fighter_2_ctrl_time,
                  "Sig_Str_head": fighter_2_sig_head,
                  "Sig_Str_body": fighter_2_sig_body,
                  "Sig_Str_leg": fighter_2_sig_leg,
                  "Sig_Str_distance": fighter_2_sig_distance,
                  "Sig_Str_clinch": fighter_2_sig_clinch,
                  "Sig_Str_ground": fighter_2_sig_ground,}
    
    if fighter_1_outcome == "W":
        winner = "red"
    elif fighter_2_outcome == "W":
        winner = "blue"
    elif fighter_1_outcome == "NC":
        winner = "NC"
    elif fighter_1_outcome == "D":
        winner = "Draw"
    else:
        winner = "None"
        
    fight_info = {"red_name" : fighter_1_name,
                  "blue_name" : fighter_2_name,
                  "weight_class" : weight_class,
                  "event": event_name,
                  "winner" : winner,
                  "method" : winner_method,
                  "round": end_round,
                  "Time": end_time,
                  "red_Significant_strikes": fighter_1_sig_str,
                  "red_Total_strikes": fighter_1_total_str,
                  "red_KDs": fighter_1_KDs,
                  "red_TDs": fighter_1_TDs,
                  "red_Sub_Atts": fighter_1_sub_atts,
                  "red_Reversals": fighter_1_reversals,
                  "red_Control_time": fighter_1_ctrl_time,
                  "red_Sig_Str_head": fighter_1_sig_head,
                  "red_Sig_Str_body": fighter_1_sig_body,
                  "red_Sig_Str_leg": fighter_1_sig_leg,
                  "red_Sig_Str_distance": fighter_1_sig_distance,
                  "red_Sig_Str_clinch": fighter_1_sig_clinch,
                  "red_Sig_Str_ground": fighter_1_sig_ground,
                  "blue_Significant_strikes": fighter_2_sig_str,
                  "blue_Total_strikes": fighter_2_total_str,
                  "blue_KDs": fighter_2_KDs,
                  "blue_TDs": fighter_2_TDs,
                  "blue_Sub_Atts": fighter_2_sub_atts,
                  "blue_Reversals": fighter_2_reversals,
                  "blue_Control_time": fighter_2_ctrl_time,
                  "blue_Sig_Str_head": fighter_2_sig_head,
                  "blue_Sig_Str_body": fighter_2_sig_body,
                  "blue_Sig_Str_leg": fighter_2_sig_leg,
                  "blue_Sig_Str_distance": fighter_2_sig_distance,
                  "blue_Sig_Str_clinch": fighter_2_sig_clinch,
                  "blue_Sig_Str_ground": fighter_2_sig_ground,
                  "url": fight_url}
        
    fighter_data.append(fighter_1_info)    
    fighter_data.append(fighter_2_info)
    full_fight_data.append(fight_info)

http://ufcstats.com/fight-details/860be3cc82895177
http://ufcstats.com/fight-details/9fe62ea683decc55
http://ufcstats.com/fight-details/69167b996ce7a7e4
http://ufcstats.com/fight-details/98842f17dfe4b4ef
http://ufcstats.com/fight-details/587d373f18f6c8c0
http://ufcstats.com/fight-details/e078d18fa9ed100d
http://ufcstats.com/fight-details/88256dd5a9aef1cc
http://ufcstats.com/fight-details/d44d735e5079bfb8
http://ufcstats.com/fight-details/1fe58cdab57d233b
http://ufcstats.com/fight-details/85f920746c58d0a6
http://ufcstats.com/fight-details/6d7eecf4acecda2a
http://ufcstats.com/fight-details/acec5f4c6fee1ae2
http://ufcstats.com/fight-details/61d9eff5fa4c8578
http://ufcstats.com/fight-details/7151ae256e6c181b


In [19]:
new_fights_df = pd.DataFrame(full_fight_data)
new_fights_df

,red_name,blue_name,weight_class,event,winner,method,round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url
0,Paulo Costa,Marvin Vettori,LIGHT HEAVYWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,blue,Decision - Unanimous,ROUND: 5,TIME: 5:00,163 of 260,172 of 269,0,1 of 1,0,0,0:47,82 of 174,67 of 72,14 of 14,154 of 249,5 of 5,4 of 6,190 of 346,214 of 373,0,1 of 5,0,0,3:27,154 of 307,33 of 36,3 of 3,168 of 316,21 of 29,1 of 1,http://ufcstats.com/fight-details/860be3cc8289...
1,Grant Dawson,Ricky Glenn,LIGHTWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,Draw,Decision - Majority,ROUND: 3,TIME: 5:00,24 of 31,71 of 85,0,3 of 11,0,0,8:26,12 of 17,11 of 13,1 of 1,3 of 7,2 of 2,19 of 22,28 of 41,88 of 104,0,0 of 0,1,0,4:30,19 of 26,7 of 12,2 of 3,8 of 19,1 of 1,19 of 21,http://ufcstats.com/fight-details/9fe62ea683de...
2,Jessica-Rose Clark,Joselyne Edwards,WOMEN'S BANTAMWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,18 of 27,54 of 74,0,5 of 11,0,0,11:40,9 of 17,5 of 5,4 of 5,10 of 19,4 of 4,4 of 4,30 of 48,72 of 94,0,0 of 0,0,0,1:38,19 of 33,7 of 9,4 of 6,23 of 41,3 of 3,4 of 4,http://ufcstats.com/fight-details/69167b996ce7...
3,Alex Caceres,SeungWoo Choi,FEATHERWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,red,Submission,ROUND: 2,TIME: 3:31,46 of 64,54 of 72,0,0 of 0,1,0,0:49,22 of 33,13 of 17,11 of 14,37 of 54,9 of 10,0 of 0,33 of 98,38 of 104,1,0 of 0,0,0,0:15,18 of 78,11 of 16,4 of 4,26 of 85,3 of 3,4 of 10,http://ufcstats.com/fight-details/98842f17dfe4...
4,Francisco Trinaldo,Dwight Grant,WELTERWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,red,Decision - Split,ROUND: 3,TIME: 5:00,36 of 70,61 of 101,0,1 of 3,0,0,4:45,20 of 50,11 of 14,5 of 6,26 of 57,4 of 5,6 of 8,31 of 133,41 of 145,0,1 of 1,0,0,0:11,20 of 114,7 of 14,4 of 5,28 of 128,1 of 1,2 of 4,http://ufcstats.com/fight-details/587d373f18f6...
5,Nicolae Negumereanu,Ike Villanueva,LIGHT HEAVYWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,red,KO/TKO,ROUND: 1,TIME: 1:18,11 of 21,11 of 21,0,0 of 1,0,0,0:20,8 of 18,0 of 0,3 of 3,7 of 14,1 of 1,3 of 6,7 of 21,7 of 21,0,0 of 1,0,0,0:00,5 of 19,2 of 2,0 of 0,7 of 20,0 of 1,0 of 0,http://ufcstats.com/fight-details/e078d18fa9ed...
6,Junyong Park,Gregory Rodrigues,MIDDLEWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,blue,KO/TKO,ROUND: 2,TIME: 3:13,56 of 109,59 of 112,0,0 of 0,0,0,0:29,50 of 103,0 of 0,6 of 6,54 of 106,2 of 3,0 of 0,68 of 90,75 of 98,0,2 of 4,0,1,3:20,53 of 73,15 of 16,0 of 1,57 of 77,3 of 3,8 of 10,http://ufcstats.com/fight-details/88256dd5a9ae...
7,Mason Jones,David Onama,LIGHTWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,87 of 157,116 of 191,0,8 of 14,0,0,9:01,65 of 133,20 of 22,2 of 2,69 of 130,10 of 18,8 of 9,90 of 153,121 of 186,0,0 of 1,0,0,0:11,80 of 142,8 of 9,2 of 2,70 of 128,19 of 23,1 of 2,http://ufcstats.com/fight-details/d44d735e5079...
8,Tabatha Ricci,Maria Oliveira,WOMEN'S STRAWWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,62 of 135,103 of 187,0,5 of 11,0,0,6:38,42 of 109,12 of 16,8 of 10,29 of 98,5 of 5,28 of 32,66 of 178,90 of 207,0,0 of 0,0,0,0:01,34 of 128,18 of 33,14 of 17,61 of 172,4 of 4,1 of 2,http://ufcstats.com/fight-details/1fe58cdab57d...
9,Jamie Pickett,Laureano Staropoli,MIDDLEWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,47 of 96,58 of 108,0,2 of 2,0,0,2:37,23 of 64,23 of 30,1 of 2,26 of 72,21 of 23,0 of 1,25 of 62,43 of 80,0,1 of 9,0,0,5:26,14 of 48,8 of 11,3 of 3,19 of 54,6 of 8,0 of 0,http://ufcstats.com/fight-details/8

In [20]:
all_fights_df = pd.read_csv("../data/rolling_fights_data.csv")
all_fights_df = all_fights_df.drop(columns="Unnamed: 0")
all_fights_df

,red_name,blue_name,weight_class,event,winner,method,round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url
0,Aspen Ladd,Norma Dumont,WOMEN'S FEATHERWEIGHT BOUT,UFC Fight Night: Ladd vs. Dumont,blue,Decision - Unanimous,ROUND: 5,TIME: 5:00,33 of 110,86 of 166,0,0 of 5,0,1,7:32,28 of 103,2 of 3,3 of 4,17 of 85,8 of 12,8 of 13,65 of 149,113 of 201,0,1 of 1,0,0,1:47,48 of 129,15 of 18,2 of 2,51 of 134,14 of 15,0 of 0,http://ufcstats.com/fight-details/330e67abac6e...
1,Andrei Arlovski,Carlos Felipe,HEAVYWEIGHT BOUT,UFC Fight Night: Ladd vs. Dumont,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,83 of 172,84 of 173,0,0 of 0,0,0,0:00,49 of 133,11 of 15,23 of 24,81 of 170,2 of 2,0 of 0,73 of 172,73 of 172,0,0 of 0,0,0,1:22,53 of 145,15 of 21,5 of 6,54 of 148,12 of 14,7 of 10,http://ufcstats.com/fight-details/188b63ec6ee1...
2,Jim Miller,Erick Gonzalez,LIGHTWEIGHT BOUT,UFC Fight Night: Ladd vs. Dumont,red,KO/TKO,ROUND: 2,TIME: 0:14,32 of 48,43 of 60,1,1 of 2,0,0,1:55,25 of 41,4 of 4,3 of 3,18 of 30,8 of 10,6 of 8,14 of 37,22 of 45,0,1 of 2,0,0,1:02,10 of 30,3 of 5,1 of 2,11 of 33,2 of 2,1 of 2,http://ufcstats.com/fight-details/d9ca00b3c62a...
3,Manon Fiorot,Mayra Bueno Silva,WOMEN'S FLYWEIGHT BOUT,UFC Fight Night: Ladd vs. Dumont,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,91 of 267,112 of 291,0,2 of 6,0,0,4:05,47 of 218,37 of 42,7 of 7,79 of 245,4 of 8,8 of 14,48 of 111,92 of 156,0,0 of 0,0,0,0:04,13 of 55,21 of 32,14 of 24,39 of 100,7 of 9,2 of 2,http://ufcstats.com/fight-details/b193e2317b21...
4,Nate Landwehr,Ludovit Klein,FEATHERWEIGHT BOUT,UFC Fight Night: Ladd vs. Dumont,red,Submission,ROUND: 3,TIME: 2:22,71 of 130,103 of 163,0,1 of 2,2,0,3:46,41 of 85,18 of 30,12 of 15,45 of 93,26 of 37,0 of 0,42 of 85,44 of 87,0,0 of 2,0,0,0:32,20 of 55,18 of 25,4 of 5,39 of 80,3 of 5,0 of 0,http://ufcstats.com/fight-details/a2e1116a9fe8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6296,Orlando Wiet,Robert Lucarelli,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,ROUND: 1,TIME: 2:50,8 of 12,11 of 15,0,0 of 0,0,0,--,7 of 11,1 of 1,0 of 0,1 of 3,0 of 0,7 of 9,2 of 6,2 of 6,0,1 of 1,1,0,--,1 of 2,0 of 1,1 of 3,2 of 6,0 of 0,0 of 0,http://ufcstats.com/fight-details/3b020d4914b4...
6297,Frank Hamaker,Thaddeus Luster,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,ROUND: 1,TIME: 4:52,2 of 3,14 of 15,0,1 of 1,3,1,--,2 of 3,0 of 0,0 of 0,1 of 1,0 of 0,1 of 2,0 of 0,0 of 0,0,0 of 1,0,0,--,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/d917c8c7461b...
6298,Johnny Rhodes,David Levicki,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,ROUND: 1,TIME: 12:13,11 of 17,74 of 86,0,1 of 1,0,0,--,9 of 15,1 of 1,1 of 1,1 of 1,1 of 1,9 of 15,4 of 5,95 of 102,0,0 of 0,0,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1,http://ufcstats.com/fight-details/ccee020be2e8...
6299,Patrick Smith,Ray Wizard,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,ROUND: 1,TIME: 0:58,1 of 1,1 of 1,0,0 of 1,1,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,1 of 1,2 of 2,0,0 of 0,0,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [21]:
fight_frames = [new_fights_df, all_fights_df]
rolling_fights_df = pd.concat(fight_frames)
rolling_fights_df

,red_name,blue_name,weight_class,event,winner,method,round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url
0,Paulo Costa,Marvin Vettori,LIGHT HEAVYWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,blue,Decision - Unanimous,ROUND: 5,TIME: 5:00,163 of 260,172 of 269,0,1 of 1,0,0,0:47,82 of 174,67 of 72,14 of 14,154 of 249,5 of 5,4 of 6,190 of 346,214 of 373,0,1 of 5,0,0,3:27,154 of 307,33 of 36,3 of 3,168 of 316,21 of 29,1 of 1,http://ufcstats.com/fight-details/860be3cc8289...
1,Grant Dawson,Ricky Glenn,LIGHTWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,Draw,Decision - Majority,ROUND: 3,TIME: 5:00,24 of 31,71 of 85,0,3 of 11,0,0,8:26,12 of 17,11 of 13,1 of 1,3 of 7,2 of 2,19 of 22,28 of 41,88 of 104,0,0 of 0,1,0,4:30,19 of 26,7 of 12,2 of 3,8 of 19,1 of 1,19 of 21,http://ufcstats.com/fight-details/9fe62ea683de...
2,Jessica-Rose Clark,Joselyne Edwards,WOMEN'S BANTAMWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,18 of 27,54 of 74,0,5 of 11,0,0,11:40,9 of 17,5 of 5,4 of 5,10 of 19,4 of 4,4 of 4,30 of 48,72 of 94,0,0 of 0,0,0,1:38,19 of 33,7 of 9,4 of 6,23 of 41,3 of 3,4 of 4,http://ufcstats.com/fight-details/69167b996ce7...
3,Alex Caceres,SeungWoo Choi,FEATHERWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,red,Submission,ROUND: 2,TIME: 3:31,46 of 64,54 of 72,0,0 of 0,1,0,0:49,22 of 33,13 of 17,11 of 14,37 of 54,9 of 10,0 of 0,33 of 98,38 of 104,1,0 of 0,0,0,0:15,18 of 78,11 of 16,4 of 4,26 of 85,3 of 3,4 of 10,http://ufcstats.com/fight-details/98842f17dfe4...
4,Francisco Trinaldo,Dwight Grant,WELTERWEIGHT BOUT,UFC Fight Night: Costa vs. Vettori,red,Decision - Split,ROUND: 3,TIME: 5:00,36 of 70,61 of 101,0,1 of 3,0,0,4:45,20 of 50,11 of 14,5 of 6,26 of 57,4 of 5,6 of 8,31 of 133,41 of 145,0,1 of 1,0,0,0:11,20 of 114,7 of 14,4 of 5,28 of 128,1 of 1,2 of 4,http://ufcstats.com/fight-details/587d373f18f6...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6296,Orlando Wiet,Robert Lucarelli,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,ROUND: 1,TIME: 2:50,8 of 12,11 of 15,0,0 of 0,0,0,--,7 of 11,1 of 1,0 of 0,1 of 3,0 of 0,7 of 9,2 of 6,2 of 6,0,1 of 1,1,0,--,1 of 2,0 of 1,1 of 3,2 of 6,0 of 0,0 of 0,http://ufcstats.com/fight-details/3b020d4914b4...
6297,Frank Hamaker,Thaddeus Luster,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,ROUND: 1,TIME: 4:52,2 of 3,14 of 15,0,1 of 1,3,1,--,2 of 3,0 of 0,0 of 0,1 of 1,0 of 0,1 of 2,0 of 0,0 of 0,0,0 of 1,0,0,--,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/d917c8c7461b...
6298,Johnny Rhodes,David Levicki,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,ROUND: 1,TIME: 12:13,11 of 17,74 of 86,0,1 of 1,0,0,--,9 of 15,1 of 1,1 of 1,1 of 1,1 of 1,9 of 15,4 of 5,95 of 102,0,0 of 0,0,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1,http://ufcstats.com/fight-details/ccee020be2e8...
6299,Patrick Smith,Ray Wizard,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,ROUND: 1,TIME: 0:58,1 of 1,1 of 1,0,0 of 1,1,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,1 of 1,2 of 2,0,0 of 0,0,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [22]:
rolling_fights_df.to_csv("../data/rolling_fights_data.csv")

In [23]:
rolling_fight_urls = rolling_fights_df["url"]
rolling_fight_url_df = pd.DataFrame(rolling_fight_urls)
rolling_fight_url_df

,url
0,http://ufcstats.com/fight-details/860be3cc8289...
1,http://ufcstats.com/fight-details/9fe62ea683de...
2,http://ufcstats.com/fight-details/69167b996ce7...
3,http://ufcstats.com/fight-details/98842f17dfe4...
4,http://ufcstats.com/fight-details/587d373f18f6...
...,...
6296,http://ufcstats.com/fight-details/3b020d4914b4...
6297,http://ufcstats.com/fight-details/d917c8c7461b...
6298,http://ufcstats.com/fight-details/ccee020be2e8...
6299,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [24]:
rolling_fight_url_df.to_csv("../data/rolling_fight_urls.csv")

In [25]:
future_event_urls

['http://ufcstats.com/event-details/3974fa35c917af1d']

In [26]:
future_fight_urls = []
future_event_data = []

for event in future_event_urls:
    browser.visit(event)
    event_soup = bs(browser.html, "html.parser")
    hrefs = event_soup.find_all(class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    
    
    event_date = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    event_name = browser.find_by_xpath("/html/body/section/div/h2/span").text
    event_location = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    
    event_info = {"event": event_name,
                 "date": event_date,
                 "location": event_location,
                 "event_url": event}
    
    future_event_data.append(event_info)
    
      
    for cols in hrefs:
        href = cols["data-link"]        
       
        if href not in future_fight_urls:
            future_fight_urls.append(href)
            
future_fight_urls

['http://ufcstats.com/fight-details/919fb76d2f3e5827',
 'http://ufcstats.com/fight-details/18edff5447989b3f',
 'http://ufcstats.com/fight-details/5ad4c0eae7512878',
 'http://ufcstats.com/fight-details/13e3e41e0c6f4e04',
 'http://ufcstats.com/fight-details/987cdcbc809e210d',
 'http://ufcstats.com/fight-details/58b19ba734fe93a3',
 'http://ufcstats.com/fight-details/d84968d7e9fcb4f3',
 'http://ufcstats.com/fight-details/41cdfea7df63003f',
 'http://ufcstats.com/fight-details/768c86e307295944',
 'http://ufcstats.com/fight-details/efc6af0151f72b87',
 'http://ufcstats.com/fight-details/92c49cb075f612d7',
 'http://ufcstats.com/fight-details/156f2425a937b22f',
 'http://ufcstats.com/fight-details/4a5e5232346993c7',
 'http://ufcstats.com/fight-details/4cdd5761107e464f',
 'http://ufcstats.com/fight-details/6163c18411aca830']

In [27]:
future_fighter_data = []
future_fight_data = []

for fight_url in future_fight_urls:
    
    browser.visit(fight_url)
    print(fight_url)
    
    event_name = browser.find_by_xpath("/html/body/section/div/h2/a").text
    weight_class = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/i").text
    
    fighter_1_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[1]/div/h3/a").text
    fighter_1_corner_color = "red"
    
    fighter_2_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[2]/div/h3/a").text
    fighter_2_corner_color = "blue"
      
    fighter_1_info = {"name" : fighter_1_name,
                  "corner" : fighter_1_corner_color,
                  "weight_class": weight_class,
                  "opponent" : fighter_2_name,
                  "event" : event_name,
                 "outcome" : "tbd",
                 "method" : "tbd",
                  "round": 0,
                  "Time": "0:00",
                  "Significant_strikes": "0 of 0",
                  "Total_strikes": "0 of 0",
                  "KDs": 0,
                  "TDs": "0 of 0",
                  "Sub_Atts": 0,
                  "Reversals": 0,
                  "Control_time": "0:00",
                  "Sig_Str_head": "0 of 0",
                  "Sig_Str_body": "0 of 0",
                  "Sig_Str_leg": "0 of 0",
                  "Sig_Str_distance": "0 of 0",
                  "Sig_Str_clinch": "0 of 0",
                  "Sig_Str_ground": "0 of 0",}
    
    fighter_2_info = {"name" : fighter_2_name,
                  "corner" : fighter_2_corner_color,
                  "weight_class": weight_class,
                  "opponent" : fighter_1_name,
                  "event" : event_name,
                 "outcome" : "tbd",
                 "method" : "tbd",
                  "round": "0",
                  "Time": "0:00",
                  "Significant_strikes": "0 of 0",
                  "Total_strikes": "0 of 0",
                  "KDs": 0,
                  "TDs": "0 of 0",
                  "Sub_Atts": 0,
                  "Reversals": 0,
                  "Control_time": "0:00",
                  "Sig_Str_head": "0 of 0",
                  "Sig_Str_body": "0 of 0",
                  "Sig_Str_leg": "0 of 0",
                  "Sig_Str_distance": "0 of 0",
                  "Sig_Str_clinch": "0 of 0",
                  "Sig_Str_ground": "0 of 0",}
    
    fight_info = {"red_name" : fighter_1_name,
                  "blue_name" : fighter_2_name,
                  "weight_class" : weight_class,
                  "event": event_name,
                  "winner" : "tbd",
                  "method" : "tbd",
                  "round": "0",
                  "Time": "0:00",
                  "red_Significant_strikes": "0 of 0",
                  "red_Total_strikes": "0 of 0",
                  "red_KDs": 0,
                  "red_TDs": "0 of 0",
                  "red_Sub_Atts": 0,
                  "red_Reversals": 0,
                  "red_Control_time": "0:00",
                  "red_Sig_Str_head": "0 of 0",
                  "red_Sig_Str_body": "0 of 0",
                  "red_Sig_Str_leg": "0 of 0",
                  "red_Sig_Str_distance": "0 of 0",
                  "red_Sig_Str_clinch": "0 of 0",
                  "red_Sig_Str_ground": "0 of 0",
                  "blue_Significant_strikes": "0 of 0",
                  "blue_Total_strikes": "0 of 0",
                  "blue_KDs": 0,
                  "blue_TDs": "0 of 0",
                  "blue_Sub_Atts": 0,
                  "blue_Reversals": 0,
                  "blue_Control_time": "0:00",
                  "blue_Sig_Str_head": "0 of 0",
                  "blue_Sig_Str_body": "0 of 0",
                  "blue_Sig_Str_leg": "0 of 0",
                  "blue_Sig_Str_distance": "0 of 0",
                  "blue_Sig_Str_clinch": "0 of 0",
                  "blue_Sig_Str_ground": "0 of 0",
                  "url": fight_url}
        
    future_fighter_data.append(fighter_1_info)    
    future_fighter_data.append(fighter_2_info)
    future_fight_data.append(fight_info)

http://ufcstats.com/fight-details/919fb76d2f3e5827
http://ufcstats.com/fight-details/18edff5447989b3f
http://ufcstats.com/fight-details/5ad4c0eae7512878
http://ufcstats.com/fight-details/13e3e41e0c6f4e04
http://ufcstats.com/fight-details/987cdcbc809e210d
http://ufcstats.com/fight-details/58b19ba734fe93a3
http://ufcstats.com/fight-details/d84968d7e9fcb4f3
http://ufcstats.com/fight-details/41cdfea7df63003f
http://ufcstats.com/fight-details/768c86e307295944
http://ufcstats.com/fight-details/efc6af0151f72b87
http://ufcstats.com/fight-details/92c49cb075f612d7
http://ufcstats.com/fight-details/156f2425a937b22f
http://ufcstats.com/fight-details/4a5e5232346993c7
http://ufcstats.com/fight-details/4cdd5761107e464f
http://ufcstats.com/fight-details/6163c18411aca830


In [28]:
future_fight_df = pd.DataFrame(future_fight_data)
future_fight_df.to_csv("../data/future_fights_data.csv")
future_fight_df

,red_name,blue_name,weight_class,event,winner,method,round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url
0,Jan Blachowicz,Glover Teixeira,UFC LIGHT HEAVYWEIGHT TITLE BOUT,UFC 267: Blachowicz vs. Teixeira,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/919fb76d2f3e...
1,Petr Yan,Cory Sandhagen,UFC INTERIM BANTAMWEIGHT TITLE BOUT,UFC 267: Blachowicz vs. Teixeira,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/18edff544798...
2,Islam Makhachev,Dan Hooker,LIGHTWEIGHT BOUT,UFC 267: Blachowicz vs. Teixeira,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/5ad4c0eae751...
3,Alexander Volkov,Marcin Tybura,HEAVYWEIGHT BOUT,UFC 267: Blachowicz vs. Teixeira,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/13e3e41e0c6f...
4,Li Jingliang,Khamzat Chimaev,WELTERWEIGHT BOUT,UFC 267: Blachowicz vs. Teixeira,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/987cdcbc809e...
5,Magomed Ankalaev,Volkan Oezdemir,LIGHT HEAVYWEIGHT BOUT,UFC 267: Blachowicz vs. Teixeira,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/58b19ba734fe...
6,Amanda Ribas,Virna Jandiroba,WOMEN'S STRAWWEIGHT BOUT,UFC 267: Blachowicz vs. Teixeira,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/d84968d7e9fc...
7,Ricardo Ramos,Zubaira Tukhugov,FEATHERWEIGHT BOUT,UFC 267: Blachowicz vs. Teixeira,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/41cdfea7df63...
8,Albert Duraev,Roman Kopylov,MIDDLEWEIGHT BOUT,UFC 267: Blachowicz vs. Teixeira,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/768c86e30729...
9,Elizeu Zaleski dos Santos,Benoit Saint-Denis,WELTERWEIGHT BOUT,UFC 267: Blachowicz vs. Teixeira,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/efc6af0151f7...


In [29]:
# UfC fighters are sorted alphabetically by last name with each letter containing a page of fighters
letters = "abcdefghijklmnopqrstuvwxyz"

# Getting URLs of each letter page
urls = []
for letter in letters:
    url = f"http://ufcstats.com/statistics/fighters?char={letter}&page=all"
    urls.append(url)
urls

['http://ufcstats.com/statistics/fighters?char=a&page=all',
 'http://ufcstats.com/statistics/fighters?char=b&page=all',
 'http://ufcstats.com/statistics/fighters?char=c&page=all',
 'http://ufcstats.com/statistics/fighters?char=d&page=all',
 'http://ufcstats.com/statistics/fighters?char=e&page=all',
 'http://ufcstats.com/statistics/fighters?char=f&page=all',
 'http://ufcstats.com/statistics/fighters?char=g&page=all',
 'http://ufcstats.com/statistics/fighters?char=h&page=all',
 'http://ufcstats.com/statistics/fighters?char=i&page=all',
 'http://ufcstats.com/statistics/fighters?char=j&page=all',
 'http://ufcstats.com/statistics/fighters?char=k&page=all',
 'http://ufcstats.com/statistics/fighters?char=l&page=all',
 'http://ufcstats.com/statistics/fighters?char=m&page=all',
 'http://ufcstats.com/statistics/fighters?char=n&page=all',
 'http://ufcstats.com/statistics/fighters?char=o&page=all',
 'http://ufcstats.com/statistics/fighters?char=p&page=all',
 'http://ufcstats.com/statistics/fighter

In [30]:
fighter_urls_df = pd.read_csv("../data/fighter_hrefs.csv")
fighter_urls_df = fighter_urls_df.drop(columns="Unnamed: 0")
fighter_urls_df

,hrefs
0,http://ufcstats.com/fighter-details/ffe9703408...
1,http://ufcstats.com/fighter-details/ffe9703408...
2,http://ufcstats.com/fighter-details/ffe9703408...
3,http://ufcstats.com/fighter-details/19c591115a...
4,http://ufcstats.com/fighter-details/19c591115a...
...,...
3796,http://ufcstats.com/fighter-details/be124bdd60...
3797,http://ufcstats.com/fighter-details/02d808afb9...
3798,http://ufcstats.com/fighter-details/1291dd6b8a...
3799,http://ufcstats.com/fighter-details/523af801b3...


In [31]:
fighter_urls = fighter_urls_df["hrefs"].tolist()
fighter_urls

['http://ufcstats.com/fighter-details/ffe9703408fb5964',
 'http://ufcstats.com/fighter-details/ffe9703408fb5964',
 'http://ufcstats.com/fighter-details/ffe9703408fb5964',
 'http://ufcstats.com/fighter-details/19c591115a58982c',
 'http://ufcstats.com/fighter-details/19c591115a58982c',
 'http://ufcstats.com/fighter-details/19c591115a58982c',
 'http://ufcstats.com/fighter-details/a2ccf35a88346a91',
 'http://ufcstats.com/fighter-details/a2ccf35a88346a91',
 'http://ufcstats.com/fighter-details/a2ccf35a88346a91',
 'http://ufcstats.com/fighter-details/bd381cde8426077a',
 'http://ufcstats.com/fighter-details/bd381cde8426077a',
 'http://ufcstats.com/fighter-details/bd381cde8426077a',
 'http://ufcstats.com/fighter-details/c2299ec916bc7c56',
 'http://ufcstats.com/fighter-details/c2299ec916bc7c56',
 'http://ufcstats.com/fighter-details/c2299ec916bc7c56',
 'http://ufcstats.com/fighter-details/8d20d76af3f213ce',
 'http://ufcstats.com/fighter-details/8d20d76af3f213ce',
 'http://ufcstats.com/fighter-d

In [32]:
# Getting links to each fighter in each letter page
hrefs = []
for url in urls:
    browser.visit(url)
    fighter_soup = bs(browser.html, "html.parser")
    rows = fighter_soup.find_all(class_="b-link b-link_style_black")
    
    
    for cols in rows:
        href = cols["href"]
        if href not in fighter_urls:
            hrefs.append(href)
hrefs

[]

In [33]:
# Creating dataframe and saving fighter URLs 
new_href_dict = {"hrefs" : hrefs}
new_href_df = pd.DataFrame(new_href_dict)
new_href_df

,hrefs


In [34]:
fighter_frames = [new_href_df, fighter_urls_df]
rolling_fighter_df = pd.concat(fighter_frames)
rolling_fighter_df

,hrefs
0,http://ufcstats.com/fighter-details/ffe9703408...
1,http://ufcstats.com/fighter-details/ffe9703408...
2,http://ufcstats.com/fighter-details/ffe9703408...
3,http://ufcstats.com/fighter-details/19c591115a...
4,http://ufcstats.com/fighter-details/19c591115a...
...,...
3796,http://ufcstats.com/fighter-details/be124bdd60...
3797,http://ufcstats.com/fighter-details/02d808afb9...
3798,http://ufcstats.com/fighter-details/1291dd6b8a...
3799,http://ufcstats.com/fighter-details/523af801b3...


In [35]:
rolling_fighter_df.to_csv("../data/fighter_hrefs.csv")

In [36]:
all_fighters_data = []
for link in hrefs:
    browser.visit(link)
    print(link)
    
    name = browser.find_by_xpath("/html/body/section/div/h2/span[1]").text
    ufc_record = browser.find_by_xpath("/html/body/section/div/h2/span[2]").text
    nickname = browser.find_by_xpath("/html/body/section/div/p").text

    height = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    weight = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    reach = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[3]").text
    stance = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[4]").text
    DOB = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[5]").text
    SLpM = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[1]/ul/li[1]").text
    Str_Acc = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[1]/ul/li[2]").text
    SApM = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[1]/ul/li[3]").text
    Str_Def = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[1]/ul/li[4]").text
    TD_Avg = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[2]/ul/li[2]").text
    TD_Acc = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[2]/ul/li[3]").text
    TD_Def = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[2]/ul/li[4]").text
    Sub_Avg = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[2]/ul/li[5]").text
    
    fighter_data = {"name" : name,
                "ufc_record" : ufc_record,
                "nickname" : nickname,
                "height" : height,
                "reach": reach,    
               "weight" : weight,
               "stance" : stance,
               "DOB" : DOB,
               "SLpM" : SLpM,
               "SApM" : SApM,
               "Str_Acc" : Str_Acc,
               "Str_Def" : Str_Def,
               "TD_Avg" : TD_Avg,
               "TD_Acc" : TD_Acc,
               "TD_Def" : TD_Def,
               "Sub_Avg" : Sub_Avg}
    
    all_fighters_data.append(fighter_data)
    
all_fighters_data

[]

In [37]:
fighter_df = pd.DataFrame(all_fighters_data)
fighter_df

""


In [38]:
# Stripping out unnecessary text from columns
fighter_df["name"] = fighter_df["name"].str.strip("\n ")
fighter_df["ufc_record"] = fighter_df["ufc_record"].str.strip("RECORD:")
fighter_df["height"] = fighter_df["height"].str.strip("HEIGHT:")
fighter_df["reach"] = fighter_df["reach"].str.strip("REACH:")
fighter_df["weight"] = fighter_df["weight"].str.strip("WEIGHT: lbs.")
fighter_df["stance"] = fighter_df["stance"].str.strip("STANCE:")
fighter_df["DOB"] = fighter_df["DOB"].str.strip("DOB:")
fighter_df["SLpM"] = fighter_df["SLpM"].str.strip("SLpM:")
fighter_df["SApM"] = fighter_df["SApM"].str.strip("SApM:")
fighter_df["Str_Acc"] = fighter_df["Str_Acc"].str.strip("Str. Acc.: %")
fighter_df["Str_Def"] = fighter_df["Str_Def"].str.strip("Str. Def: %")
fighter_df["TD_Avg"] = fighter_df["TD_Avg"].str.strip("TD Avg.:")
fighter_df["TD_Acc"] = fighter_df["TD_Acc"].str.strip("TD Acc.: %")
fighter_df["TD_Def"] = fighter_df["TD_Def"].str.strip("TD Def.: %")
fighter_df["Sub_Avg"] = fighter_df["Sub_Avg"].str.strip("Sub. Avg.:")
fighter_df

KeyError: 'name'

In [ ]:
fighter_df.info()

In [ ]:
# convert objects to floats
fighter_df["SLpM"] = fighter_df["SLpM"].astype(float, errors = 'raise')
fighter_df["SApM"] = fighter_df["SApM"].astype(float, errors = 'raise')
fighter_df["Str_Acc"] = fighter_df["Str_Acc"].astype(float, errors = 'raise')
fighter_df["Str_Def"] = fighter_df["Str_Def"].astype(float, errors = 'raise')
fighter_df["TD_Avg"] = fighter_df["TD_Avg"].astype(float, errors = 'raise')
fighter_df["TD_Acc"] = fighter_df["TD_Acc"].astype(float, errors = 'raise')
fighter_df["TD_Def"] = fighter_df["TD_Def"].astype(float, errors = 'raise')
fighter_df["Sub_Avg"] = fighter_df["Sub_Avg"].astype(float, errors = 'raise')
fighter_df.info()

In [ ]:
fighter_df = fighter_df.rename(columns={"Str_Acc":"Str_Acc_pct",
                                       "Str_Def":"Str_Def_pct",
                                       "TD_Acc":"TD_Acc_pct",
                                       "TD_Def":"TD_Def_pct"})
fighter_df = fighter_df.drop_duplicates()
fighter_df

In [ ]:
existing_fighter_df = pd.read_csv("../data/career_fighter_stats.csv")
existing_fighter_df = existing_fighter_df.drop(columns="Unnamed: 0")
existing_fighter_df

In [ ]:
all_fighter_frames = [fighter_df, existing_fighter_df]
all_fighter_df = pd.concat(all_fighter_frames)
all_fighter_df

In [ ]:
all_fighter_df.drop_duplicates()
all_fighter_df

In [ ]:
all_fighter_df.to_csv("../data/career_fighter_stats.csv")